In [ ]:
import tensorflow as tf
import numpy as np

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

nb_classes = 3 #class의 개수입니다.

print(x_data.shape)
print(y_data.shape)


(8, 4)
(8, 3)


In [ ]:
#Weight and bias setting
W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.03892944,  0.03141014,  1.2816032 ],
       [-0.61452633,  0.0303982 ,  0.7161511 ],
       [ 0.24425435,  0.39938837,  0.4733473 ],
       [ 0.63442904,  0.24472974, -0.4186666 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.9323183,  2.3536994, -0.2319128], dtype=float32)>


In [ ]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[3.0545916e-03 6.3297319e-01 3.6397216e-01]
 [6.2163942e-03 6.1961436e-01 3.7416923e-01]
 [1.3935872e-02 6.3235778e-01 3.5370633e-01]
 [1.0976086e-02 4.5657694e-01 5.3244698e-01]
 [1.8310765e-04 3.7333003e-01 6.2648684e-01]
 [1.7404554e-02 9.5580465e-01 2.6790753e-02]
 [8.0399209e-04 6.8026924e-01 3.1892684e-01]
 [5.1685155e-04 6.5919131e-01 3.4029189e-01]], shape=(8, 3), dtype=float32)


In [ ]:
# Softmax onehot test
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype=np.float32)

print(hypothesis(sample_db))

tf.Tensor([[3.2904343e-05 2.0095252e-03 9.9795759e-01]], shape=(1, 3), dtype=float32)


In [ ]:
def cost_fn(X, Y):
    logits = hypothesis(X)
    cost = -tf.reduce_sum(Y * tf.math.log(logits), axis=1)
    cost_mean = tf.reduce_mean(cost)
    
    return cost_mean

print(cost_fn(x_data, y_data))

tf.Tensor(2.442648, shape=(), dtype=float32)


In [ ]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [ ]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

print(grad_fn(x_data, y_data))

[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[-0.23498651,  0.28302228, -0.04803579],
       [-1.6147788 ,  0.7744212 ,  0.84035754],
       [-1.5981534 ,  0.87667525,  0.7214782 ],
       [-1.5950129 ,  0.87274384,  0.7222692 ]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.24336356,  0.2512647 , -0.00790114], dtype=float32)>]


In [ ]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 1.346818
Loss at epoch 100: 0.652493
Loss at epoch 200: 0.580762
Loss at epoch 300: 0.523590
Loss at epoch 400: 0.471579
Loss at epoch 500: 0.422210
Loss at epoch 600: 0.374426
Loss at epoch 700: 0.328021
Loss at epoch 800: 0.285072
Loss at epoch 900: 0.258306
Loss at epoch 1000: 0.244498
Loss at epoch 1100: 0.232231
Loss at epoch 1200: 0.221125
Loss at epoch 1300: 0.211015
Loss at epoch 1400: 0.201766
Loss at epoch 1500: 0.193271
Loss at epoch 1600: 0.185441
Loss at epoch 1700: 0.178199
Loss at epoch 1800: 0.171482
Loss at epoch 1900: 0.165235
Loss at epoch 2000: 0.159411


In [ ]:
b = hypothesis(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data

tf.Tensor(
[[9.1915107e-09 7.0366586e-05 9.9992967e-01]
 [5.3648133e-04 4.7331009e-02 9.5213246e-01]
 [4.1658952e-09 9.6879706e-02 9.0312022e-01]
 [1.8671566e-06 9.1204733e-01 8.7950796e-02]
 [1.5994181e-01 8.3323961e-01 6.8186368e-03]
 [8.4571652e-02 9.1542834e-01 2.2203164e-08]
 [8.2102346e-01 1.7897597e-01 4.5955818e-07]
 [9.7058916e-01 2.9410781e-02 1.2652970e-09]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


In [ ]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name='weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis=1))        
        return cost
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)            
            return grads
    
    def fit(self, X, Y, epochs=2000, verbose=500):
        optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)